In [1]:
import sys, os, time, shutil, random
from pathlib import Path
_cwd = os.getcwd()
os.chdir(Path(_cwd)/'..')
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
# %load_ext autoreload
# %autoreload 2
# %pdb
from itertools import product, cycle
from heapq import heapify, heappush, heappop
import h5py as hf
# import cv2
# import omnifig as fig
# import numpy as np
# np.set_printoptions(linewidth=120)
# import pickle
# from tabulate import tabulate
# from tqdm.notebook import tqdm
# import torch
# from torch import nn
# import torch.nn.functional as F
# import torch.distributions as distrib
# import torch.multiprocessing as mp
# # import torchvision.models
# from torch.utils.data import Dataset, DataLoader, TensorDataset
# import networkx as nx

# %matplotlib notebook
# import matplotlib.pyplot as plt
# #plt.switch_backend('Qt5Agg') #('Qt5Agg')
# import matplotlib.patches as mpatches
# import matplotlib.pyplot as plt
# import matplotlib.cm
# from matplotlib import animation
# import matplotlib as mpl
# from matplotlib.collections import PatchCollection
# # mpl.rc('image', cmap='gray')
# import seaborn as sns
# import pandas as pd
# from sklearn.preprocessing import normalize

# fig.initialize()

# import omnilearn as learn
# from omnilearn import models
# from omnilearn import util
# from omnilearn.data import InterventionSamplerBase
# from omnilearn import viz as viz_util

# from sklearn.cluster import MeanShift, estimate_bandwidth, DBSCAN, KMeans
# from sklearn.datasets import make_blobs
# from sklearn.decomposition import PCA

# from src import sample_full_interventions, response_mat, conditioned_reponses
# from src.responses import sample_full_interventions

# # import gpumap
# from c3linearize import linearize, class_graph

# fig.get_config().silence();

In [2]:
root = Path(r'C:\Users\anwan\Documents\workspace') / 'lat_resp_models'

In [4]:
ms = list(root.glob('**/ckpt100000/results/metrics.pth.tar'))
len(ms)

76

In [27]:
names = [m.parents[1].name for m in ms]
len(names)

113

In [29]:
print(names)

['3ds_vae1_d-conv_d24-s1_210523-164723', '3ds_vae1_d-conv_d24-s2_210523-164722', '3ds_vae1_d-conv_s1_210523-164722', '3ds_vae1_d-conv_s2_210523-164723', '3ds_vae2_d-conv_d24-s1_210523-164722', '3ds_vae2_d-conv_d24-s2_210523-164722', '3ds_vae2_d-conv_s1_210523-164723', '3ds_vae2_d-conv_s2_210523-164722', '3ds_vae4_d-conv_d24-s1_210523-164722', '3ds_vae4_d-conv_d24-s2_210523-164722', '3ds_vae4_d-conv_s1_210523-164723', '3ds_vae4_d-conv_s2_210523-164722', '3ds_vae8_d-conv_d24-s1_210523-164723', '3ds_vae8_d-conv_d24-s2_210523-164722', '3ds_vae8_d-conv_s1_210523-164722', '3ds_vae8_d-conv_s2_210523-164722', 'celeba_vae12_e-c12_d-c12_d64_210521-051200', 'celeba_vae1_e-c12_d-c12_d64-s2_210521-013413', 'celeba_vae2_e-c12_d-c12_d64_210521-013413', 'celeba_vae4_e-c12_d-c12_d64_210521-013413', 'celeba_vae8_e-c12_d-c12_d64_210521-051200', 'toy_vae1_d-conv_210531-001122', 'toy_vae1_d-conv_strd-trans-lrp0001_210531-001122', 'toy_vae8_d-conv_210531-001122', 'toy_vae8_d-conv_strd-trans-lrp0001_210531-0

In [30]:
tmpl = 'unbuffer fig eval-multiple-metrics eval/3ds --saveroot /home/fleeb/saved_models/lat_resp_models/ --run-name {}'

In [31]:
print('\n'.join(tmpl.format(x) for x in names))

unbuffer fig eval-multiple-metrics eval/3ds --saveroot /home/fleeb/saved_models/lat_resp_models/ --run-name 3ds_vae1_d-conv_d24-s1_210523-164723
unbuffer fig eval-multiple-metrics eval/3ds --saveroot /home/fleeb/saved_models/lat_resp_models/ --run-name 3ds_vae1_d-conv_d24-s2_210523-164722
unbuffer fig eval-multiple-metrics eval/3ds --saveroot /home/fleeb/saved_models/lat_resp_models/ --run-name 3ds_vae1_d-conv_s1_210523-164722
unbuffer fig eval-multiple-metrics eval/3ds --saveroot /home/fleeb/saved_models/lat_resp_models/ --run-name 3ds_vae1_d-conv_s2_210523-164723
unbuffer fig eval-multiple-metrics eval/3ds --saveroot /home/fleeb/saved_models/lat_resp_models/ --run-name 3ds_vae2_d-conv_d24-s1_210523-164722
unbuffer fig eval-multiple-metrics eval/3ds --saveroot /home/fleeb/saved_models/lat_resp_models/ --run-name 3ds_vae2_d-conv_d24-s2_210523-164722
unbuffer fig eval-multiple-metrics eval/3ds --saveroot /home/fleeb/saved_models/lat_resp_models/ --run-name 3ds_vae2_d-conv_s1_210523-1647

In [15]:
# for p in ms:
#     r = p.parents[1]
#     if not (r/'last').exists():
#         os.symlink(str(p.parents[0]), str(r/'last'), target_is_directory=True)
#         results = r/'last'/'results'
#         results.mkdir()

#         for o in r.glob('*.pth.tar'):
#             shutil.move(str(o), str(results/o.name))
# #         break
#     else:
#         print(f'skipping: {r}')
    

skipping: C:\Users\anwan\Documents\workspace\lat_resp_models\3ds_vae1_d-conv_d24-s1_210523-164723
skipping: C:\Users\anwan\Documents\workspace\lat_resp_models\3ds_vae1_d-conv_d24-s2_210523-164722
skipping: C:\Users\anwan\Documents\workspace\lat_resp_models\3ds_vae1_d-conv_s1_210523-164722


In [16]:
# mets = list(root.glob('**/metrics.pth.tar'))
# len(mets)

54

In [18]:
# for m in mets:
#     if m.exists():
#         shutil.move(str(m), str(m.parents[0]/'old_metrics.pth.tar'))

In [5]:
rnames = '''3ds_vae1_d-conv_s1_210523-164722	3ds_vae1_d-conv_d24-s1_210523-164723
3ds_vae1_d-conv_s2_210523-164723	3ds_vae1_d-conv_d24-s2_210523-164722
3ds_vae2_d-conv_s1_210523-164723	3ds_vae2_d-conv_d24-s1_210523-164722
3ds_vae2_d-conv_s2_210523-164722	3ds_vae2_d-conv_d24-s2_210523-164722
3ds_vae4_d-conv_s1_210523-164723	3ds_vae4_d-conv_d24-s1_210523-164722
3ds_vae4_d-conv_s2_210523-164722	3ds_vae4_d-conv_d24-s2_210523-164722
3ds_vae8_d-conv_s1_210523-164722	3ds_vae8_d-conv_d24-s1_210523-164723
3ds_vae8_d-conv_s2_210523-164722	3ds_vae8_d-conv_d24-s2_210523-164722'''.replace('\t', '\n').split('\n')

rnames = '''toy_vae1_e-c12_d-c12_d6-ch128-strd-trans_210522-020221
toy_vae1_e-c12_d-c12_d8-ch128-strd-trans_210522-020221
toy_vae1_e-c12_d-c12_d10-ch128-strd-trans_210522-020222
toy_vae1_e-c12_d-c12_ch128-strd-trans_210522-015012
toy_vae1_e-c12_d-c12_d16-ch128-strd-trans_210522-020223
toy_vae1_e-c12_d-c12_d32-ch128-strd-trans_210522-020222'''.split('\n')

rnames = '''real_vae1_e-c12_d-c12_d6-ch128-strd-trans_210522-020221
real_vae1_e-c12_d-c12_d8-ch128-strd-trans_210522-020222
real_vae1_e-c12_d-c12_d10-ch128-strd-trans_210522-020221
real_vae1_e-c12_d-c12_d12-ch128-strd-trans_210522-020222
real_vae1_e-c12_d-c12_d16-ch128-strd-trans_210522-020222
real_vae1_e-c12_d-c12_d32-ch128-strd-trans_210522-020221'''.split('\n')

In [10]:
rnames = '''268M    real_vae1_d-conv_strd-trans-lrp0001-ch128_210531-005631
267M    real_vae1_d-conv_strd-trans-lrp001-ch128_210531-005631
268M    real_vae2_d-conv_strd-trans-lrp0001-ch128_210531-005631
267M    real_vae2_d-conv_strd-trans-lrp001-ch128_210531-005631
268M    real_vae4_d-conv_strd-trans-lrp0001-ch128_210531-005631
267M    real_vae4_d-conv_strd-trans-lrp001-ch128_210531-005631
268M    real_vae8_d-conv_strd-trans-lrp0001-ch128_210531-005632
267M    real_vae8_d-conv_strd-trans-lrp001-ch128_210531-005632
88M     toy_vae1_d-conv_210531-001122
107M    toy_vae1_d-conv_strd-trans-lrp0001_210531-001122
263M    toy_vae1_d-conv_strd-trans-lrp0001-ch128_210531-001122
260M    toy_vae1_d-conv_strd-trans-lrp001-ch128_210531-001123
262M    toy_vae2_d-conv_strd-trans-lrp0001-ch128_210531-005631
261M    toy_vae2_d-conv_strd-trans-lrp001-ch128_210531-005632
262M    toy_vae4_d-conv_strd-trans-lrp0001-ch128_210531-005631
259M    toy_vae4_d-conv_strd-trans-lrp001-ch128_210531-005630
86M     toy_vae8_d-conv_210531-001122
105M    toy_vae8_d-conv_strd-trans-lrp0001_210531-001123
261M    toy_vae8_d-conv_strd-trans-lrp0001-ch128_210531-001123
253M    toy_vae8_d-conv_strd-trans-lrp001-ch128_210531-001122'''.split('\n')
rnames = [r.split(' ')[-1] for r in rnames]
len(rnames)

20

In [11]:
tmpl = 'fleeb@$STOKKUR:/is/ei/fleeb/workspace/chome/trained_nets/{}'
cmd = 'rsync -rvz {} .'

In [12]:
print(cmd.format(' '.join(tmpl.format(n) for n in rnames)))

rsync -rvz fleeb@$STOKKUR:/is/ei/fleeb/workspace/chome/trained_nets/real_vae1_d-conv_strd-trans-lrp0001-ch128_210531-005631 fleeb@$STOKKUR:/is/ei/fleeb/workspace/chome/trained_nets/real_vae1_d-conv_strd-trans-lrp001-ch128_210531-005631 fleeb@$STOKKUR:/is/ei/fleeb/workspace/chome/trained_nets/real_vae2_d-conv_strd-trans-lrp0001-ch128_210531-005631 fleeb@$STOKKUR:/is/ei/fleeb/workspace/chome/trained_nets/real_vae2_d-conv_strd-trans-lrp001-ch128_210531-005631 fleeb@$STOKKUR:/is/ei/fleeb/workspace/chome/trained_nets/real_vae4_d-conv_strd-trans-lrp0001-ch128_210531-005631 fleeb@$STOKKUR:/is/ei/fleeb/workspace/chome/trained_nets/real_vae4_d-conv_strd-trans-lrp001-ch128_210531-005631 fleeb@$STOKKUR:/is/ei/fleeb/workspace/chome/trained_nets/real_vae8_d-conv_strd-trans-lrp0001-ch128_210531-005632 fleeb@$STOKKUR:/is/ei/fleeb/workspace/chome/trained_nets/real_vae8_d-conv_strd-trans-lrp001-ch128_210531-005632 fleeb@$STOKKUR:/is/ei/fleeb/workspace/chome/trained_nets/toy_vae1_d-conv_210531-001122 fl

In [4]:
print('\n'.join(tmpl.format(n) for n in rnames))

rsync -rvz fleeb@$STOKKUR:/is/ei/fleeb/workspace/chome/trained_nets/3ds_vae1_d-conv_s1_210523-164722 .
rsync -rvz fleeb@$STOKKUR:/is/ei/fleeb/workspace/chome/trained_nets/3ds_vae1_d-conv_d24-s1_210523-164723 .
rsync -rvz fleeb@$STOKKUR:/is/ei/fleeb/workspace/chome/trained_nets/3ds_vae1_d-conv_s2_210523-164723 .
rsync -rvz fleeb@$STOKKUR:/is/ei/fleeb/workspace/chome/trained_nets/3ds_vae1_d-conv_d24-s2_210523-164722 .
rsync -rvz fleeb@$STOKKUR:/is/ei/fleeb/workspace/chome/trained_nets/3ds_vae2_d-conv_s1_210523-164723 .
rsync -rvz fleeb@$STOKKUR:/is/ei/fleeb/workspace/chome/trained_nets/3ds_vae2_d-conv_d24-s1_210523-164722 .
rsync -rvz fleeb@$STOKKUR:/is/ei/fleeb/workspace/chome/trained_nets/3ds_vae2_d-conv_s2_210523-164722 .
rsync -rvz fleeb@$STOKKUR:/is/ei/fleeb/workspace/chome/trained_nets/3ds_vae2_d-conv_d24-s2_210523-164722 .
rsync -rvz fleeb@$STOKKUR:/is/ei/fleeb/workspace/chome/trained_nets/3ds_vae4_d-conv_s1_210523-164723 .
rsync -rvz fleeb@$STOKKUR:/is/ei/fleeb/workspace/chome/tr

In [11]:
tmpl = 'unbuffer fig eval-multiple-metrics eval/mpi --cat real --run-name {}'
# tmpl = 'unbuffer fig eval-multiple-metrics eval/3ds --run-name {}'

In [12]:
print('\n'.join(tmpl.format(n) for n in rnames))

unbuffer fig eval-multiple-metrics eval/mpi --cat real --run-name real_vae1_e-c12_d-c12_d6-ch128-strd-trans_210522-020221
unbuffer fig eval-multiple-metrics eval/mpi --cat real --run-name real_vae1_e-c12_d-c12_d8-ch128-strd-trans_210522-020222
unbuffer fig eval-multiple-metrics eval/mpi --cat real --run-name real_vae1_e-c12_d-c12_d10-ch128-strd-trans_210522-020221
unbuffer fig eval-multiple-metrics eval/mpi --cat real --run-name real_vae1_e-c12_d-c12_d12-ch128-strd-trans_210522-020222
unbuffer fig eval-multiple-metrics eval/mpi --cat real --run-name real_vae1_e-c12_d-c12_d16-ch128-strd-trans_210522-020222
unbuffer fig eval-multiple-metrics eval/mpi --cat real --run-name real_vae1_e-c12_d-c12_d32-ch128-strd-trans_210522-020221


In [3]:
root = Path(r'C:\Users\anwan\Documents\workspace')


In [4]:
univ= list(root.glob('**/config.yaml'))
len(univ)

935

In [5]:
univ

[WindowsPath('C:/Users/anwan/Documents/workspace/extrap/3ds_ae_d-conv_210404-132852/config.yaml'),
 WindowsPath('C:/Users/anwan/Documents/workspace/extrap/3ds_ae_e-c12_d-c12_ohu_210517-234751/config.yaml'),
 WindowsPath('C:/Users/anwan/Documents/workspace/extrap/3ds_ae_e-c12_d-c12_sh_210517-232435/config.yaml'),
 WindowsPath('C:/Users/anwan/Documents/workspace/extrap/3ds_ae_e-c12_d-c12_xas_210424-022530/config.yaml'),
 WindowsPath('C:/Users/anwan/Documents/workspace/extrap/3ds_ae_e-c12_d-c12_xas_210509-025523/config.yaml'),
 WindowsPath('C:/Users/anwan/Documents/workspace/extrap/3ds_ae_e-c12_d-c12_xa_210424-022637/config.yaml'),
 WindowsPath('C:/Users/anwan/Documents/workspace/extrap/3ds_ae_e-c12_d-c12_xa_210509-032547/config.yaml'),
 WindowsPath('C:/Users/anwan/Documents/workspace/extrap/3ds_ae_e-c12_d-s12_210411-165818/config.yaml'),
 WindowsPath('C:/Users/anwan/Documents/workspace/extrap/3ds_ae_e-c12_d-s12_d24-lr0001_210414-171331/config.yaml'),
 WindowsPath('C:/Users/anwan/Document

In [6]:
keep = set()

In [9]:
keep.update('''rfd_ae_e-c16_d-c16_d16-strd-trans-lrp00005-bs64-ch256_210524-213908
rfd_ae_e-c16_d-s16_d16-strd-trans-lrp00005-bs64-ch256_210524-161708
rfd_ae_e-c16_d-s16_d16-strd-trans-lrp00005-bs64-ch256-nosplit_210525-122527
rfd_vae1_e-c16_d-c16_d16-strd-trans-lrp00005-bs64-ch256_210524-214205
rfd_vae2_e-c16_d-c16_d16-strd-trans-lrp00005-bs64-ch256_210524-214633
rfd_vae1_e-l16_d-l16_d16-strd-trans-lrp00005-bs64-ch256_210524-214809
rfd_wae10_e-c16_d-c16_d16-strd-trans-lrp00005-bs64-ch256_210524-215009'''.split('\n')
           )

keep.update('''3ds_ae0_e-c12_d-c12_mish_210125-050038	toy_ae_e-c12_d-c12_mish_210125-222234	sim_ae_e-c12_d-c12_mish_210126-035850	real_ae_e-c12_d-c12_mish_210126-035928	celeba_ae0_e-c16_d-c16_mish_210125-132401	rfd_ae_e-c16_d-c16_d16-strd-trans-lrp00005-bs64-ch256_210524-213908
3ds_vae1_e-c12_d-c12_mish_210125-140254	toy_vae1_e-c12_d-c12_mish_210126-122537	sim_vae1_e-c12_d-c12_mish_210126-123641	real_vae1_e-c12_d-c12_mish_210126-123340	celeba_vae1_e-c16_d-c16_mish_210125-132401	rfd_vae1_e-c16_d-c16_d16-strd-trans-lrp00005-bs64-ch256_210524-214205
3ds_vae2_e-c12_d-c12_mish_210125-140255	toy_vae2_e-c12_d-c12_mish_210126-122537	sim_vae2_e-c12_d-c12_mish_210126-123642	real_vae2_e-c12_d-c12_mish_210126-123340	celeba_vae2_e-c16_d-c16_mish-lrp0003_210201-101817	rfd_vae2_e-c16_d-c16_d16-strd-trans-lrp00005-bs64-ch256_210524-214633
3ds_vae4_e-c12_d-c12_mish_210125-140302	toy_vae4_e-c12_d-c12_mish_210127-231812	sim_vae4_e-c12_d-c12_mish_210127-231257	real_vae4_e-c12_d-c12_mish_210127-231852	celeba_vae4_e-c16_d-c16_mish-lrp0004_210201-101817	
3ds_vae8_e-c12_d-c12_mish_210125-140550	toy_vae8_e-c12_d-c12_mish_210127-231816	sim_vae8_e-c12_d-c12_mish_210127-231257	real_vae8_e-c12_d-c12_mish_210127-231853		
3ds_wae10_e-c12_d-c12_mish_210125-140253	toy_wae10_e-c12_d-c12_mish_210126-122537	sim_wae10_e-c12_d-c12_mish_210126-123641	real_wae10_e-c12_d-c12_mish_210126-123340	celeba_wae10_e-c16_d-c16_mish_210201-000430	rfd_wae10_e-c16_d-c16_d16-strd-trans-lrp00005-bs64-ch256_210524-215009
3ds_wae100_e-c12_d-c12_mish_210125-221143	toy_wae100_e-c12_d-c12_mish_210126-122537	sim_wae100_e-c12_d-c12_mish_210126-123641	real_wae100_e-c12_d-c12_mish_210126-123340		
3ds_wae1000_e-c12_d-c12_mish_210125-221142	toy_wae1000_e-c12_d-c12_mish_210126-122537	sim_wae1000_e-c12_d-c12_mish_210126-123641	real_wae1000_e-c12_d-c12_mish_210126-123339		
3ds_ae_e-c12_d-s2_mish_210131-150237	toy_ae_e-c12_d-s2_mish_210127-231820	sim_ae_e-c12_d-s2_mish_210127-231304	real_ae_e-c12_d-s2_mish_210127-231829		
3ds_ae_e-c12_d-s2_nosplit-mish_210131-150234	toy_ae_e-c12_d-s2_nosplit-mish_210131-150234	sim_ae_e-c12_d-s2_nosplit-mish_210127-231312	real_ae_e-c12_d-s2_nosplit-mish_210131-150234		
3ds_ae_e-c12_d-s3_mish_210201-021841	toy_ae_e-c12_d-s3_mish_210127-231823	sim_ae_e-c12_d-s3_mish_210127-231307	real_ae_e-c12_d-s3_mish_210127-231829		
3ds_ae_e-c12_d-s3_nosplit-mish_210131-150234	toy_ae_e-c12_d-s3_nosplit-mish_210131-150236	sim_ae_e-c12_d-s3_nosplit-mish_210127-231312	real_ae_e-c12_d-s3_nosplit-mish_210131-150234		
3ds_ae0_e-c12_d-s4_mish_210125-140253	toy_ae_e-c12_d-s4_mish_210126-122536	sim_ae_e-c12_d-s4_mish_210127-231308	real_ae_e-c12_d-s4_mish_210126-123339		
3ds_ae_e-c12_d-s4_nosplit-mish_210131-150234	toy_ae_e-c12_d-s4_nosplit-mish_210131-150235	sim_ae_e-c12_d-s4_nosplit-mish_210127-231315	real_ae_e-c12_d-s4_nosplit-mish_210131-150234		
3ds_ae0_e-c12_d-s6_mish_210125-140253	toy_ae_e-c12_d-s6_mish_210126-122536	sim_ae_e-c12_d-s6_mish_210127-231308	real_ae_e-c12_d-s6_mish_210126-123339		
3ds_ae_e-c12_d-s6_nosplit-mish_210131-150234	toy_ae_e-c12_d-s6_nosplit-mish_210131-150236	sim_ae_e-c12_d-s6_nosplit-mish_210127-231313	real_ae_e-c12_d-s6_nosplit-mish_210131-150236		
3ds_ae0_e-c12_d-s12_mish_210125-050335	toy_ae_e-c12_d-s12_mish_210125-222233	sim_ae_e-c12_d-s12_mish_210126-035851	real_ae_e-c12_d-s12_mish_210127-231831	celeba_ae0_e-c16_d-s16_mish_210125-132401	rfd_ae_e-c16_d-s16_d16-strd-trans-lrp00005-bs64-ch256_210524-161708
3ds_ae_e-c12_d-s12_nosplit-mish_210131-150236	toy_ae_e-c12_d-s12_nosplit-mish_210131-150234	sim_ae_e-c12_d-s12_nosplit-mish_210127-231314	real_ae_e-c12_d-s12_nosplit-mish_210127-231851	celeba_ae0_e-c16_d-s16_nosplit-mish_210125-132401	rfd_ae_e-c16_d-s16_d16-strd-trans-lrp00005-bs64-ch256-nosplit_210525-122527
3ds_vae1_e-l2_d-l2_mish_210131-150235	toy_vae1_e-l2_d-l2_mish_210131-150234	sim_vae1_e-l2_d-l2_mish_210131-150238	real_vae1_e-l2_d-l2_mish_210131-150240		
3ds_vae1_e-l3_d-l3_mish_210131-150235	toy_vae1_e-l3_d-l3_mish_210131-150234	sim_vae1_e-l3_d-l3_mish_210131-150236	real_vae1_e-l3_d-l3_mish_210131-150236		
3ds_vae1_e-l4_d-l4_mish_210125-134914	toy_vae1_e-l4_d-l4_mish_210131-150235	sim_vae1_e-l4_d-l4_mish_210127-231449	real_vae1_e-l4_d-l4_mish_210131-150236		
3ds_vae1_e-l6_d-l6_mish_210125-140021	toy_vae1_e-l6_d-l6_mish_210131-150235	sim_vae1_e-l6_d-l6_mish_210127-231447	real_vae1_e-l6_d-l6_mish_210131-150236		
3ds_vae1_e-l12_d-l12_mish_210125-140249	toy_vae1_e-l12_d-l12_mish_210131-150237	sim_vae1_e-l12_d-l12_mish_210127-231450	real_vae1_e-l12_d-l12_mish_210127-231847	celeba_vae1_e-l16_d-l16_mish-lrp0004_210201-034237	rfd_vae1_e-l16_d-l16_d16-strd-trans-lrp00005-bs64-ch256_210524-214809
					
3ds_ae0_e-c12_d-c12_mish_210125-050038	toy_ae_e-c12_d-c12_mish_210125-222234	sim_ae_e-c12_d-c12_mish_210126-035850	real_ae_e-c12_d-c12_mish_210126-035928	celeba_ae0_e-c16_d-c16_mish_210125-132401	
3ds_ae_e-c12_d-c12_mish-seed2_210131-230456					
3ds_ae_e-c12_d-c12_mish-seed3_210131-230456					
3ds_ae_e-c12_d-c12_mish-seed4_210131-230456					
3ds_ae_e-c12_d-c12_mish-seed5_210131-230456					
3ds_vae1_e-c12_d-c12_mish_210125-140254	toy_vae1_e-c12_d-c12_mish_210126-122537	sim_vae1_e-c12_d-c12_mish_210126-123641	real_vae1_e-c12_d-c12_mish_210126-123340	celeba_vae1_e-c16_d-c16_mish_210125-132401	
3ds_vae1_e-c12_d-c12_mish-seed2_210131-230457					
3ds_vae1_e-c12_d-c12_mish-seed3_210131-230501					
3ds_vae1_e-c12_d-c12_mish-seed4_210131-230501					
3ds_vae1_e-c12_d-c12_mish-seed5_210131-230500					
3ds_vae2_e-c12_d-c12_mish_210125-140255	toy_vae2_e-c12_d-c12_mish_210126-122537	sim_vae2_e-c12_d-c12_mish_210126-123642	real_vae2_e-c12_d-c12_mish_210126-123340		
3ds_vae2_e-c12_d-c12_mish-seed2_210131-230501					
3ds_vae2_e-c12_d-c12_mish-seed3_210131-230459					
3ds_vae2_e-c12_d-c12_mish-seed4_210131-230503					
3ds_vae2_e-c12_d-c12_mish-seed5_210131-230501					
3ds_wae10_e-c12_d-c12_mish_210125-140253	toy_wae10_e-c12_d-c12_mish_210126-122537	sim_wae10_e-c12_d-c12_mish_210126-123641	real_wae10_e-c12_d-c12_mish_210126-123340		
3ds_wae10_e-c12_d-c12_mish-seed2_210201-021010					
3ds_wae10_e-c12_d-c12_mish-seed3_210201-021013					
3ds_wae10_e-c12_d-c12_mish-seed4_210201-021011					
3ds_wae10_e-c12_d-c12_mish-seed5_210201-021840					
3ds_wae100_e-c12_d-c12_mish_210125-221143	toy_wae100_e-c12_d-c12_mish_210126-122537	sim_wae100_e-c12_d-c12_mish_210126-123641	real_wae100_e-c12_d-c12_mish_210126-123340		
3ds_wae100_e-c12_d-c12_mish-seed2_210131-230501					
3ds_wae100_e-c12_d-c12_mish-seed3_210131-230503					
3ds_wae100_e-c12_d-c12_mish-seed4_210131-230503					
3ds_wae100_e-c12_d-c12_mish-seed5_210131-230503					
3ds_ae0_e-c12_d-s4_mish_210125-140253	toy_ae_e-c12_d-s4_mish_210126-122536	sim_ae_e-c12_d-s4_mish_210127-231308	real_ae_e-c12_d-s4_mish_210126-123339		
3ds_ae_e-c12_d-s4_mish-seed2_210131-230504					
3ds_ae_e-c12_d-s4_mish-seed3_210131-230506					
3ds_ae_e-c12_d-s4_mish-seed4_210131-230506					
3ds_ae_e-c12_d-s4_mish-seed5_210131-230505					
3ds_ae0_e-c12_d-s6_mish_210125-140253	toy_ae_e-c12_d-s6_mish_210126-122536	sim_ae_e-c12_d-s6_mish_210127-231308	real_ae_e-c12_d-s6_mish_210126-123339		
3ds_ae_e-c12_d-s6_mish-seed2_210131-230505					
3ds_ae_e-c12_d-s6_mish-seed3_210131-230503					
3ds_ae_e-c12_d-s6_mish-seed4_210131-230505					
3ds_ae_e-c12_d-s6_mish-seed5_210131-230504					
3ds_ae0_e-c12_d-s12_mish_210125-050335	toy_ae_e-c12_d-s12_mish_210125-222233	sim_ae_e-c12_d-s12_mish_210126-035851	real_ae_e-c12_d-s12_mish_210127-231831	celeba_ae0_e-c16_d-s16_mish_210125-132401	
3ds_ae_e-c12_d-s12_seed3-mish_210126-004805	toy_ae_e-c12_d-s12_mish-seed2_210126-122537	sim_ae_e-c12_d-s12_mish-seed2_210126-123640	real_ae_e-c12_d-s12_mish-seed2_210131-150234		
3ds_ae_e-c12_d-s12_mish-seed2_210131-150235	toy_ae_e-c12_d-s12_mish-seed3_210126-122537	sim_ae_e-c12_d-s12_mish-seed3_210126-123641	real_ae_e-c12_d-s12_mish-seed3_210131-150234		
3ds_ae_e-c12_d-s12_mish-seed5_210131-150235	toy_ae_e-c12_d-s12_mish-seed4_210126-122537	sim_ae_e-c12_d-s12_mish-seed4_210126-123641	real_ae_e-c12_d-s12_mish-seed4_210131-150234		
3ds_ae_e-c12_d-s12_mish-seed8_210201-035456	toy_ae_e-c12_d-s12_mish-seed5_210126-122537	sim_ae_e-c12_d-s12_mish-seed5_210126-123641	real_ae_e-c12_d-s12_mish-seed5_210131-150235		
3ds_ae_e-c12_d-s12_mish-seed6_210201-035456					
3ds_ae_e-c12_d-s12_nosplit-mish_210131-150236	toy_ae_e-c12_d-s12_nosplit-mish_210131-150234	sim_ae_e-c12_d-s12_nosplit-mish_210127-231314	real_ae_e-c12_d-s12_nosplit-mish_210127-231851	celeba_ae0_e-c16_d-s16_nosplit-mish_210125-132401	
3ds_ae_e-c12_d-s12_nosplit-mish-seed3_210131-230502					
3ds_ae_e-c12_d-s12_nosplit-mish-seed5_210131-230503					
3ds_ae_e-c12_d-s12_nosplit-mish-seed4_210131-230506					
3ds_ae_e-c12_d-s12_mish-nosplit-seed7_210201-035456					
3ds_vae1_e-l12_d-l12_mish_210125-140249	toy_vae1_e-l12_d-l12_mish_210131-150237	sim_vae1_e-l12_d-l12_mish_210127-231450	real_vae1_e-l12_d-l12_mish_210127-231847	celeba_vae1_e-l16_d-l16_mish_210125-132401	
3ds_vae1_e-l12_d-l12_mish-seed2_210131-230506					
3ds_vae1_e-l12_d-l12_mish-seed3_210131-230505					
3ds_vae1_e-l12_d-l12_mish-seed4_210131-230507					
3ds_vae1_e-l12_d-l12_mish-seed5_210131-230507					'''.replace('\t', '\n').split('\n'))

keep = {k for k in keep if len(k)}
len(keep)

194